In [ ]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
import numpy as np
from scipy.sparse import hstack
import utils

TESTING_MODE = False

In [ ]:
data = utils.load_data("./data/train.csv",is_train= TESTING_MODE)
print("Loaded {0} examples with {1} columns".format(data.shape[0],data.shape[1]))

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
one_hot_encoding = utils.one_hot_encoding(data,["country","region_1","region_2","province","taster_name","variety"])


In [ ]:
X = hstack([one_hot_encoding,data[["points"]]]).tocsr()
y = data["price"]
if not TESTING_MODE:
    X_test = X[175000:]
    y_test = y[175000:]
    X = X[:175000]
    y = y[:175000]

In [ ]:
# convert to <if True> if you want to search for the parameters
if False:
    params = {
        "max_depth" : [2, 5, 10, 15, 20, 25, 30, 50, 75, 100],
        "min_samples_split" : np.linspace(0.1, 1.0, 5, endpoint=True),
        "min_samples_leaf" : np.linspace(0.1, .5, 5, endpoint=True),
        "max_features" : np.linspace(0.1, 1.0, 5, endpoint=True),
    }

    dtr = DecisionTreeRegressor()
    grid_search = utils.train_grid_search(dtr, X, y, params)
    print(grid_search.best_estimator_)

In [17]:
dtr = DecisionTreeRegressor(max_depth=5, max_features=1.0,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.1,
           min_samples_split=0.1, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

dtr, scores = utils.train_kfold(dtr, X, y)

Training fold number :  0
RMSE for fold number 0 = 34.88727616095883
Training fold number :  1
RMSE for fold number 1 = 36.014820927069955
Training fold number :  2
RMSE for fold number 2 = 32.720803699891135
Training fold number :  3
RMSE for fold number 3 = 34.22039522505831
Training fold number :  4
RMSE for fold number 4 = 32.37798683998908


In [ ]:
if not TESTING_MODE:
    ids = data["id"][175000:]
    y_pred = dtr.predict(X_test)
    result = pd.DataFrame()
    result['id'] = ids
    result["price"] = y_pred
    result.to_csv("out_DT.csv")